In [1]:
import os
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

In [2]:
def list_files(dirn):
    return [os.path.join(dirn, f) for f in sorted(os.listdir(dirn))]

def split_label(lab):
    """
    given the color code of each class,
    split label images (H, W, C: RGB) into (H, W, N:number of classes)
    
    Params:
        @lab: label image
    
    Return:
        @res: the mask of every class stacked in the form of (H,W,N)
    """
    color_code = {
        'background': [255,255,255],
        'red': [200,0,0],
        'orange': [100,0,10],
        'green': [0, 100, 10],
        'blue': [0, 10, 100]
    }
    a = lab.reshape(-1,3)
    
    res = []
    for name, color in color_code.items():
        tmp = np.array([sum(c==np.array(color))==3 for c in a])
        tmp = tmp.reshape(lab.shape[:2]).astype(np.uint8)
        res.append(tmp)
    return np.stack(res, axis=2)

def split_pred(pred):
    u_label = np.unique(pred)
    res = []
    for u in u_label:
        tmp = np.array(pred == u).astype(np.uint8)
        res.append(tmp)
    return np.stack(res, axis=2)

def max_mIoU(label_mask, pred_possible_mask):
    """
    Find the maximum intersect over the union (IoU) between a mask of one label and all possible masks.
    
    
    Params:
        @label_mask: label mask (H,W,Nc)
        @pred_possible_mask: mask of the prediction (H,W,N)
    
    Return:
        @iou_list: a list of IoU value of each label class
        
    Procedures:
        1. For one label class, we calculate the intersect with all possible prediction masks.
        2. Then, we select out the prediction mask with maximum intersect as the predicted 
        masks to calculate the IoU for that class.
        3. Repeat 1,2 for every label class
        
    """
    n_label = label_mask.shape[-1]
    
    iou_list = []
    used = []
    for i in range(n_label):
        # take out a mask of one label
        one_label_mask = label_mask[:,:,i:i+1]
        # calculate the intersect with each possible mask
        counts = np.sum(one_label_mask*pred_possible_mask, axis=(0,1))
        imax = np.argmax(counts)
        while(imax in used):
            counts[imax] = 0
            imax = np.argmax(counts)
        used.append(imax)
        iou = counts[np.argmax(counts)]/np.sum(one_label_mask)
        iou_list.append(iou)
    return iou_list

In [4]:
labels = list_files("outputs/labels/")

label_dict = {}
for l in labels:
    a = io.imread(l)
    aa = split_label(a)
    label_dict[os.path.basename(l)] = aa
    
with open('outputs/label_masks.pickle', 'wb') as handle:
    pickle.dump(label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open('outputs/label_masks.pickle', 'rb') as handle:
    labels = pickle.load(handle)

In [6]:
dir_list = ["outputs/mean_shift/", 
            "outputs/k-means/k5/", 
            "outputs/k-means/k10/", 
            "outputs/min_cut/", 
            "outputs/superpixel/", 
            "outputs/test/"]

In [12]:
all_res = {}
for dirn in dir_list:
    files = [f for f in list_files(dirn) if '.png' in f or '.jpg' in f or '.bmp' in f]
    dirn_res = []
    for f in files:
        pred = io.imread(f, as_gray=True)
        pred = split_pred(pred)
        true = labels[os.path.basename(f)]
        dirn_res.append(max_mIoU(true, pred))
    all_res[dirn] = dirn_res

In [15]:
print("overall performance \t\t\t average IoU per class \t\t\t\t method")
for n, mat in all_res.items():
    m = np.array(mat)
    print(np.mean(m),'\t', np.mean(m, axis=0), '\t',n)

overall performance 			 average IoU per class 				 method
0.36424653618657826 	 [0.82912781 0.3730021  0.40570626 0.06400253 0.14939398] 	 outputs/mean_shift/
0.40484692118803245 	 [0.83030628 0.39226822 0.36578747 0.37104455 0.06482809] 	 outputs/k-means/k5/
0.3789222103289222 	 [0.74430909 0.23747528 0.2546463  0.22855035 0.42963003] 	 outputs/k-means/k10/
0.16315805636157593 	 [0.29848504 0.06932657 0.05865145 0.10335814 0.28596908] 	 outputs/min_cut/
0.5567872695803221 	 [0.69264395 0.48682731 0.49708074 0.42147785 0.6859065 ] 	 outputs/superpixel/
0.5781003618932057 	 [0.80408953 0.49427817 0.39945016 0.58076309 0.61192086] 	 outputs/test/
